In [1]:
import numpy as np
from PIL import Image
import pandas as pd

path = './train/'
target_path = './crop-train/'

target_width = 512
target_height = 512

In [2]:
df = pd.read_csv('./trainLabels.csv')

img_names = df['image']
img_label = df['level']

In [3]:
def cropImage(im, name):
    image_arr = np.array(im)
    n_row = image_arr.shape[0]
    n_col = image_arr.shape[1]
    
    s = np.sum(image_arr, axis=(2,0)) / n_col
    s = np.round(s, decimals=0)
    s = s.astype(np.int64)
    
    minVal = 10
    maxVal = (255 - minVal) * 3
    
    k = s - minVal
    s = np.clip(k, a_min=0, a_max=(maxVal))
    
    z = np.where(s != 0)[0]
    
    if z.shape[0] < 2: # image is completely black
        print("Blank image")
    else:
        right_edge = z[z.shape[0]-1]+1
        left_edge = z[0,]
        w = right_edge - left_edge
        
        top_edge = 0 # initially top and bottom edges are set as that of image
        bottom_edge = n_row
        h = bottom_edge - top_edge
        
        cropped = None
        
        if n_row > w: # there is space above and below
            # find top and bottom edges
            
            s = np.sum(image_arr, axis=(2,1)) / n_row
            s = np.round(s, decimals=0)
            s = s.astype(np.int64)
            
            s = np.clip((s - minVal), a_min=0, a_max=(maxVal))
            z = np.where(s != 0)[0]
            
            bottom_edge = z[z.shape[0]-1]+1
            top_edge = z[0,]
            h = bottom_edge - top_edge
            
        else:
            pad = int((w - h) / 2)
            newRows = np.zeros_like(image_arr[0:pad,])
            image_arr = np.vstack((newRows, image_arr))
            image_arr = np.vstack((image_arr, newRows))
            im = Image.fromarray(image_arr)
            bottom_edge = image_arr.shape[0]
            
        cropped = im.crop((left_edge, top_edge, right_edge, bottom_edge))
        cropped = cropped.resize((target_width, target_height))
        
        cropped.save(target_path + name + '.jpeg')

In [4]:
for name in img_names:
    pil_im = Image.open(path + name + ".jpeg")
    cropImage(pil_im, name)

Blank image
Blank image
Blank image
Blank image
